In [2]:
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import os
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.ticker as plticker
import pybedtools
import os
import os.path
import pandas as pd
from ast import literal_eval

amplicons = pd.read_csv("~/iCloud/dev/Sherlock-Lung/results2/amplicon-table.tsv", sep="\t", converters={"Oncogenes": literal_eval, "All genes": literal_eval, "Location": literal_eval})
amplicons.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,AA amplicon number,Feature ID,Classification,Location,Oncogenes,All genes,Complexity score,...,SBS-Novel-A,SBS-Novel-B,SBS-Novel-C,Samples,SBS80A,SBS80B,SBS80C,method,Gene_Status,ecDNA-class
0,0,0,0,1.0,114_T_amplicon1_ecDNA_1,ecDNA,"[chr12:68500116-70201087, chr12:70203797-70357...","[MDM2, YEATS4]","[BEST3, CCT2, CNOT2, CNOT2-DT, CPM, CPSF6, FRS...",0.692400,...,0.0,0.0,0.0,NSLC-0763-T01,30.0,74.0,42.0,vanilla,Oncogene,Simple
1,1,1,1,1.0,16MH0041_P2_amplicon1_Complex non-cyclic_1,Complex non-cyclic,[chr8:79721175-79775830],[HEY1],[HEY1],0.639378,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,vanilla,Oncogene,Mono-locus
2,2,2,2,2.0,16MH0041_P2_amplicon2_Complex non-cyclic_1,Complex non-cyclic,"[chr8:80929626-81282732, chr8:118266235-119218...",[],"[COLEC10, FABP5, MAL2, PAG1, SAMD12, SAMD12-AS...",1.200132,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,vanilla,No oncogene,Simple
3,3,3,3,3.0,16MH0041_P2_amplicon3_Linear amplification_1,Linear amplification,"[chr8:101221897-101318719, chr8:128428309-1284...",[],[],1.097740,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,vanilla,No gene,Simple
4,4,4,4,4.0,16MH0041_P2_amplicon4_Linear amplification_1,Linear amplification,[chr8:120491669-120685016],[],[],0.680851,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,vanilla,No gene,Mono-locus


In [48]:
amplicons = pd.read_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/amplicon-table.tsv", sep="\t", converters={"Oncogenes": literal_eval, "All genes": literal_eval, "Location": literal_eval})
amplicons = amplicons[amplicons["method"] == "rescaled"]
amplicons = amplicons[amplicons["Classification"] == "ecDNA"]
amplicons["Gene_Status"].value_counts(dropna=False)

Oncogene       161
No oncogene    131
No gene         67
Name: Gene_Status, dtype: int64

In [82]:
import csv
amplicons = pd.read_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/amplicon-table.tsv", sep="\t", converters={"Oncogenes": literal_eval, "All genes": literal_eval, "Location": literal_eval})
amplicons = amplicons[amplicons["method"] == "rescaled"]
amplicons2 = amplicons[amplicons["Classification"] != "ecDNA"]
amplicons = amplicons[amplicons["Classification"] == "ecDNA"]

#GET ENHANCER STATUS OF ECDNA
enhancer = "/Users/azhark/iCloud/dev/Sherlock-Lung/lung_enhancer.bed"
enhancers = os.listdir("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/enhancer")
enhancers = [x.replace("_enhancer.bed", "") for x in enhancers]
enhancers = set(enhancers)

status = []
for identifier, l in zip(amplicons["Feature ID"], amplicons["Gene_Status"]):
    bed = "/Users/azhark/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/temp/" + identifier + ".bed"
    if (identifier in enhancers) and (l == "No gene"):
        status.append("Enhancer")
    else:
        status.append(l)
amplicons["Gene_Status"] = status
    # with open(bed, "w") as f:
    #     writer = csv.writer(f, delimiter="\t")
    #     for line in l:
    #         l2 = line.split(":")
    #         chrom = line.split(":")[0]
    #         start = l2[1].split("-")[0]
    #         stop = l2[1].split("-")[1]
    #         writer.writerow([chrom, start, stop])

    # a = pybedtools.BedTool(bed) #ecDNA region
    # b = pybedtools.BedTool(enhancer)
    # #fraction of overlap of B (enhancer) with A has to be 1, only report a hit once, 
    # a.intersect(b, u=True, F=1).saveas("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/enhancer/" + identifier + "_enhancer.bed")
amplicons["Gene_Status"].value_counts()

Oncogene       161
No oncogene    131
Enhancer        40
No gene         27
Name: Gene_Status, dtype: int64

In [99]:
amplicons["Classification"].value_counts()

ecDNA    359
Name: Classification, dtype: int64

In [89]:
c = pd.concat([amplicons,amplicons2],ignore_index=True)
c["Classification"].value_counts()



Linear                646
No amplification      609
Complex-non-cyclic    439
ecDNA                 359
BFB                    95
Name: Classification, dtype: int64

In [92]:
c.iloc[:, 3:].head()

,AA amplicon number,Feature ID,Classification,Location,Oncogenes,All genes,Complexity score,Captured interval length,Feature median copy number,Feature maximum copy number,...,SBS-Novel-A,SBS-Novel-B,SBS-Novel-C,Samples,SBS80A,SBS80B,SBS80C,method,Gene_Status,ecDNA-class
0,3.0,114_T_amplicon3_ecDNA_1,ecDNA,"[chr12:68500201-70201087, chr12:70203797-70357...","[MDM2, YEATS4]","[BEST3, CCT2, CNOT2, CNOT2-DT, CPM, CPSF6, FRS...",0.546461,2237330.0,5.422275,6.746037,...,0.0,0.0,0.0,NSLC-0763-T01,30.0,74.0,42.0,rescaled,Oncogene,Simple
1,1.0,16MH0041_P2_amplicon1_ecDNA_1,ecDNA,[chr2:57171634-57220091],[],[],0.623371,48457.0,4.704572,4.704572,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,Enhancer,Mono-locus
2,3.0,16MH0041_P2_amplicon3_ecDNA_1,ecDNA,[chr8:106146321-106276048],[],[OXR1],0.437090,129727.0,4.641889,4.641889,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Mono-locus
3,4.0,16MH0041_P2_amplicon4_ecDNA_1,ecDNA,"[chr9:2211962-2279367, chr9:2385302-2469676, c...",[],"[VLDLR, VLDLR-AS1]",1.406928,307189.0,4.152645,5.073657,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Simple
4,4.0,17MH0022_4R_amplicon4_ecDNA_1,ecDNA,"[chr18:76288114-78160279, chr18:78160326-78437...",[],"[C18orf65, GALR1, MBP, ZNF236, ZNF236-DT, ZNF516]",0.857253,2257073.0,6.458750,7.133459,...,19795.0,0.0,0.0,17MH0022_4R,669.0,254.0,2.0,rescaled,No oncogene,Simple


In [98]:
c.iloc[:, 3:].to_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/rescaled-amplicon-table.tsv", sep="\t", index=None)

In [97]:
c["ecDNA-class"].value_counts()

Simple              1116
Mono-locus           688
Multi-chromosome     344
Name: ecDNA-class, dtype: int64

In [71]:
#amplicons = pd.read_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/amplicon-table.tsv", sep="\t", converters={"Oncogenes": literal_eval, "All genes": literal_eval, "Location": literal_eval})
amplicons = amplicons.dropna(subset=['Feature ID'])
#amplicons = amplicons[~amplicons["Feature ID"].str.contains("NA")]
id = []

for identifier, l in zip(amplicons["Feature ID"], amplicons["Location"]):
    d = identifier	
    d2 = d.split("_")
    print(len(d2), d2)
    if len(d2) == 4:
        d3 = d2[0] + "_" + d2[2] + "_" + d2[3]
    else:
        d3 = d2[0] + "_" + d2[1] + "_" + d2[3] + "_" + d2[4]  
    id.append(d3)
amplicons["ID"] = id



5 ['114', 'T', 'amplicon3', 'ecDNA', '1']
5 ['16MH0041', 'P2', 'amplicon1', 'ecDNA', '1']
5 ['16MH0041', 'P2', 'amplicon3', 'ecDNA', '1']
5 ['16MH0041', 'P2', 'amplicon4', 'ecDNA', '1']
5 ['17MH0022', '4R', 'amplicon4', 'ecDNA', '1']
5 ['2831', 'T', 'amplicon3', 'ecDNA', '1']
5 ['4540', 'T', 'amplicon2', 'ecDNA', '1']
5 ['4540', 'T', 'amplicon2', 'ecDNA', '2']
4 ['AEIUFAVJ', 'amplicon4', 'ecDNA', '1']
4 ['E00934102', 'amplicon1', 'ecDNA', '1']
4 ['E01014101', 'amplicon1', 'ecDNA', '1']
4 ['EBUS110413B', 'amplicon2', 'ecDNA', '1']
5 ['EBUS130815A', '11R', 'amplicon2', 'ecDNA', '1']
5 ['EBUS230413A', '4R', 'amplicon3', 'ecDNA', '1']
5 ['EBUS230413A', '4R', 'amplicon3', 'ecDNA', '2']
5 ['EBUS230413A', '4R', 'amplicon3', 'ecDNA', '3']
5 ['EBUS230413A', '4R', 'amplicon3', 'ecDNA', '4']
5 ['GT01008', '2016', 'amplicon5', 'ecDNA', '1']
5 ['GT01012', '2019', 'amplicon1', 'ecDNA', '1']
5 ['GT01014', '2017', 'amplicon8', 'ecDNA', '1']
5 ['GT01050', '2016', 'amplicon1', 'ecDNA', '1']
5 ['GT01050'

In [63]:
amplicons

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,AA amplicon number,Feature ID,Classification,Location,Oncogenes,All genes,Complexity score,...,SBS-Novel-B,SBS-Novel-C,Samples,SBS80A,SBS80B,SBS80C,method,Gene_Status,ecDNA-class,ID
1843,1843,1843,1843,3.0,114_T_amplicon3_ecDNA_1,ecDNA,"[chr12:68500201-70201087, chr12:70203797-70357...","[MDM2, YEATS4]","[BEST3, CCT2, CNOT2, CNOT2-DT, CPM, CPSF6, FRS...",0.546461,...,0.0,0.0,NSLC-0763-T01,30.0,74.0,42.0,rescaled,Oncogene,Simple,114_T_ecDNA_1
1846,1846,1846,1846,1.0,16MH0041_P2_amplicon1_ecDNA_1,ecDNA,[chr2:57171634-57220091],[],[],0.623371,...,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,Enhancer,Mono-locus,16MH0041_P2_ecDNA_1
1847,1847,1847,1847,3.0,16MH0041_P2_amplicon3_ecDNA_1,ecDNA,[chr8:106146321-106276048],[],[OXR1],0.437090,...,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Mono-locus,16MH0041_P2_ecDNA_1
1848,1848,1848,1848,4.0,16MH0041_P2_amplicon4_ecDNA_1,ecDNA,"[chr9:2211962-2279367, chr9:2385302-2469676, c...",[],"[VLDLR, VLDLR-AS1]",1.406928,...,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Simple,16MH0041_P2_ecDNA_1
1855,1855,1855,1855,4.0,17MH0022_4R_amplicon4_ecDNA_1,ecDNA,"[chr18:76288114-78160279, chr18:78160326-78437...",[],"[C18orf65, GALR1, MBP, ZNF236, ZNF236-DT, ZNF516]",0.857253,...,0.0,0.0,17MH0022_4R,669.0,254.0,2.0,rescaled,No oncogene,Simple,17MH0022_4R_ecDNA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,3952,3952,3952,1.0,TCGA-91-6847-01A_amplicon1_ecDNA_1,ecDNA,"[chr7:54468491-55219065, chr7:55221368-56327434]",[EGFR],"[CCT6A, CHCHD2, EGFR, EGFR-AS1, ELDR, FKBP9P1,...",0.376552,...,0.0,0.0,TCGA-91-6847-01A,254.0,185.0,0.0,rescaled,Oncogene,Simple,TCGA-91-6847-01A_ecDNA_1
3954,3954,3954,3954,10.0,TCGA-91-7771-01A_amplicon10_ecDNA_1,ecDNA,[chr20:63936964-63998062],[],"[PRPF6, SAMD10, UCKL1, UCKL1-AS1, ZNF512B]",0.505967,...,0.0,0.0,TCGA-91-7771-01A,336.0,330.0,85.0,rescaled,No oncogene,Mono-locus,TCGA-91-7771-01A_ecDNA_1
3957,3957,3957,3957,3.0,TCGA-91-7771-01A_amplicon3_ecDNA_1,ecDNA,"[chr4:29727712-29730723, chr4:32702510-3270954...",[],[],0.680872,...,0.0,0.0,TCGA-91-7771-01A,336.0,330.0,85.0,rescaled,Enhancer,Simple,TCGA-91-7771-01A_ecDNA_1
3974,3974,3974,3974,1.0,TCGA-97-8174-01A_amplicon1_ecDNA_1,ecDNA,[chr5:706712-775904],[],[ZDHHC11B],0.921171,...,0.0,0.0,TCGA-97-8174-01A,136.0,133.0,0.0,rescaled,No oncogene,Mono-locus,TCGA-97-8174-01A_ecDNA_1


In [72]:
# amplicons['Gene_Status'] = amplicons['Gene_Status'].astype('category')
# amplicons['Gene_Status'] = amplicons['Gene_Status'].cat.set_categories(["Oncogene ", "No oncogene", "Enhancer", "No gene"], ordered=True)
# amplicons.sort_values(by=["ID", 'Gene_Status'], inplace=True, ascending=True)
# amplicons2 = amplicons.drop_duplicates(subset=["ID"]) #now we are at the sample leve







In [67]:
amplicons2.iloc[:, 3:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 1843 to 3983
Data columns (total 83 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   AA amplicon number           311 non-null    float64 
 1   Feature ID                   311 non-null    object  
 2   Classification               0 non-null      category
 3   Location                     311 non-null    object  
 4   Oncogenes                    311 non-null    object  
 5   All genes                    311 non-null    object  
 6   Complexity score             311 non-null    float64 
 7   Captured interval length     311 non-null    float64 
 8   Feature median copy number   311 non-null    float64 
 9   Feature maximum copy number  311 non-null    float64 
 10  Filter flag                  311 non-null    object  
 11  Reference version            311 non-null    object  
 12  Tissue of origin             0 non-null      float64 
 13  S

In [75]:
amplicons2.iloc[:, 3:].to_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/rescaled-amplicon-table.tsv", sep="\t", index=None)

In [73]:
amplicons2.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,AA amplicon number,Feature ID,Classification,Location,Oncogenes,All genes,Complexity score,...,SBS-Novel-B,SBS-Novel-C,Samples,SBS80A,SBS80B,SBS80C,method,Gene_Status,ecDNA-class,ID
1843,1843,1843,1843,3.0,114_T_amplicon3_ecDNA_1,ecDNA,"[chr12:68500201-70201087, chr12:70203797-70357...","[MDM2, YEATS4]","[BEST3, CCT2, CNOT2, CNOT2-DT, CPM, CPSF6, FRS...",0.546461,...,0.0,0.0,NSLC-0763-T01,30.0,74.0,42.0,rescaled,NaN,Simple,114_T_ecDNA_1
1847,1847,1847,1847,3.0,16MH0041_P2_amplicon3_ecDNA_1,ecDNA,[chr8:106146321-106276048],[],[OXR1],0.437090,...,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Mono-locus,16MH0041_P2_ecDNA_1
1855,1855,1855,1855,4.0,17MH0022_4R_amplicon4_ecDNA_1,ecDNA,"[chr18:76288114-78160279, chr18:78160326-78437...",[],"[C18orf65, GALR1, MBP, ZNF236, ZNF236-DT, ZNF516]",0.857253,...,0.0,0.0,17MH0022_4R,669.0,254.0,2.0,rescaled,No oncogene,Simple,17MH0022_4R_ecDNA_1
1865,1865,1865,1865,3.0,2831_T_amplicon3_ecDNA_1,ecDNA,"[chr8:143633513-144021102, chr8:144023037-1446...",[RECQL4],"[ADCK5, ARHGAP39, BOP1, BREA2, C8orf82, CCDC16...",0.545988,...,0.0,0.0,NSLC-0772-T01,83.0,71.0,69.0,rescaled,NaN,Simple,2831_T_ecDNA_1
1887,1887,1887,1887,2.0,4540_T_amplicon2_ecDNA_1,ecDNA,"[chr16:34899771-34910546, chr16:34913438-34948...",[],[],0.659192,...,0.0,0.0,NSLC-0785-T01,150.0,109.0,0.0,rescaled,Enhancer,Simple,4540_T_ecDNA_1


In [74]:
amplicons2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 1843 to 3983
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Unnamed: 0                   311 non-null    int64   
 1   Unnamed: 0.1                 311 non-null    int64   
 2   Unnamed: 0.1.1               311 non-null    int64   
 3   AA amplicon number           311 non-null    float64 
 4   Feature ID                   311 non-null    object  
 5   Classification               311 non-null    object  
 6   Location                     311 non-null    object  
 7   Oncogenes                    311 non-null    object  
 8   All genes                    311 non-null    object  
 9   Complexity score             311 non-null    float64 
 10  Captured interval length     311 non-null    float64 
 11  Feature median copy number   311 non-null    float64 
 12  Feature maximum copy number  311 non-null    float64 
 13  F

In [39]:
amplicons["Gene_Status"].value_counts(dropna=False)

No gene        433
No oncogene    405
Oncogene       342
Name: Gene_Status, dtype: int64

In [3]:
genes = pd.read_csv("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/rescaled-genes.tsv", sep="\t")
amplicons = pd.read_csv("~/iCloud/dev/Sherlock-Lung/results2/rescaled-amplicon-table.tsv", sep="\t", converters={"Oncogenes": literal_eval, "All genes": literal_eval, "Location": literal_eval})

In [5]:
amplicons.head()

,AA amplicon number,Feature ID,Classification,Location,Oncogenes,All genes,Complexity score,Captured interval length,Feature median copy number,Feature maximum copy number,...,SBS-Novel-A,SBS-Novel-B,SBS-Novel-C,Samples,SBS80A,SBS80B,SBS80C,method,Gene_Status,ecDNA-class
0,3.0,114_T_amplicon3_ecDNA_1,ecDNA,"[chr12:68500201-70201087, chr12:70203797-70357...","[MDM2, YEATS4]","[BEST3, CCT2, CNOT2, CNOT2-DT, CPM, CPSF6, FRS...",0.546461,2237330.0,5.422275,6.746037,...,0.0,0.0,0.0,NSLC-0763-T01,30.0,74.0,42.0,rescaled,Oncogene,Simple
1,1.0,16MH0041_P2_amplicon1_ecDNA_1,ecDNA,[chr2:57171634-57220091],[],[],0.623371,48457.0,4.704572,4.704572,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,Enhancer,Mono-locus
2,3.0,16MH0041_P2_amplicon3_ecDNA_1,ecDNA,[chr8:106146321-106276048],[],[OXR1],0.437090,129727.0,4.641889,4.641889,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Mono-locus
3,4.0,16MH0041_P2_amplicon4_ecDNA_1,ecDNA,"[chr9:2211962-2279367, chr9:2385302-2469676, c...",[],"[VLDLR, VLDLR-AS1]",1.406928,307189.0,4.152645,5.073657,...,0.0,0.0,0.0,16MH0041_P2,183.0,131.0,0.0,rescaled,No oncogene,Simple
4,4.0,17MH0022_4R_amplicon4_ecDNA_1,ecDNA,"[chr18:76288114-78160279, chr18:78160326-78437...",[],"[C18orf65, GALR1, MBP, ZNF236, ZNF236-DT, ZNF516]",0.857253,2257073.0,6.458750,7.133459,...,19795.0,0.0,0.0,17MH0022_4R,669.0,254.0,2.0,rescaled,No oncogene,Simple


In [6]:
amplicons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148 entries, 0 to 2147
Data columns (total 82 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   AA amplicon number           1539 non-null   float64
 1   Feature ID                   2148 non-null   object 
 2   Classification               2148 non-null   object 
 3   Location                     2148 non-null   object 
 4   Oncogenes                    2148 non-null   object 
 5   All genes                    2148 non-null   object 
 6   Complexity score             1539 non-null   float64
 7   Captured interval length     1539 non-null   float64
 8   Feature median copy number   1539 non-null   float64
 9   Feature maximum copy number  1539 non-null   float64
 10  Filter flag                  1539 non-null   object 
 11  Reference version            2148 non-null   object 
 12  Tissue of origin             0 non-null      float64
 13  Sample type       

In [7]:
amplicons["Gene_Status"].value_counts()

No gene        1069
No oncogene     536
Oncogene        503
Enhancer         40
Name: Gene_Status, dtype: int64

In [9]:
amplicons['Feature ID'].duplicated().sum()

2